In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install emot underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 43.0 MB/s eta 0:00:00


In [5]:
import pandas as pd
import re
from bs4 import BeautifulSoup

from emot.emo_unicode import UNICODE_EMOJI 
from emot.emo_unicode import EMOTICONS_EMO 

import spacy
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize

from underthesea import word_tokenize

import functionSQL as f 

import warnings
warnings.filterwarnings('ignore')

In [16]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
%cd /content/drive/MyDrive/colab_notebook

conn, cursor = f.init()

data = f.extract_all_data(conn, cursor, 'REVIEWS')

data.head()

/content/drive/MyDrive/colab_notebook


,category,sub_category,name_product,star,title,comment
0,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,"Nope don't buy, bad quality!",I really wanted to like these!!!! A few proble...
1,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,Don't buy,These shirts look great in the picture but aft...
2,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,THE WORST,This was the worst item I ever purchased for m...
3,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,What Happened,This product definitely missed the mark. To be...
4,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,Single stitch seam/raw edge- do not buy!,"It’s a single stitch seam, this video is afte..."


### 1. Explore Data

In [8]:
data.isnull().sum() / len(data) * 100

category        0.000000
sub_category    0.000000
name_product    0.000000
star            0.000000
title           0.004344
comment         0.241095
dtype: float64

In [9]:
data = data.dropna().reset_index(drop=True)
data.isnull().sum().sum()

0

In [10]:
data = data.drop_duplicates().reset_index(drop=True)
data.head()

,category,sub_category,name_product,star,title,comment
0,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,"Nope don't buy, bad quality!",I really wanted to like these!!!! A few proble...
1,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,Don't buy,These shirts look great in the picture but aft...
2,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,THE WORST,This was the worst item I ever purchased for m...
3,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,What Happened,This product definitely missed the mark. To be...
4,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,Single stitch seam/raw edge- do not buy!,"It’s a single stitch seam, this video is afte..."


### 2. Text Processing

In [17]:
stop_words = set(stopwords.words('english'))
def remove_stopword(text): 
    words = word_tokenize(text)
    return ' '.join([word for word in words if word.lower() not in stop_words])

def lower(text):
    return text.lower().replace('\n', '')
 
def remove_tags(html): 
    soup = BeautifulSoup(html, "html.parser")
    for data in soup(['style', 'script']):
        data.decompose() # Remove tags
    return ' '.join(soup.stripped_strings)

def remove_punc(text): # consist: %, ^, &, *, (, ), {, }, [, ], ;, :, /, ?, <, >, \, |
    return re.sub(r'[^\w\s]','',text)

def remove_digit(text): 
    return re.sub("\d+", " ", text)

def converting_emojis(text):
    for x in EMOTICONS_EMO:
        text = text.replace(x, "_".join(EMOTICONS_EMO[x].replace(",","").replace(":","").split()))
        
    for x in UNICODE_EMOJI:
        text = text.replace(x, "_".join(UNICODE_EMOJI[x].replace(",","").replace(":","").split()))
            
    return text
 
def split_word(text): 
    return word_tokenize(text)
    
nlp = spacy.load("en_core_web_sm")
def lemmatize(text):
  doc = nlp(text)
  return " ".join([token.lemma_ for token in doc])

def remove_non_english_words(text):
    english_words = set(words.words())
    words_list = text.split()
    english_words_list = [word for word in words_list if word.lower() in english_words]
    cleaned_text = ' '.join(english_words_list)
    return cleaned_text

def remove_spaces(text): 
    return re.sub(r"\s+", " ", text)

In [18]:
def clean_all(text): 
    text = lower(text)
    text = remove_stopword(text)
    text = lemmatize(text)
    # text = stemming(text)
    text = remove_tags(text)
    text = remove_punc(text)
    text = remove_digit(text)
    text = converting_emojis(text)
    # text = remove_non_english_words(text)
    text = remove_spaces(text)
    return text.strip()

In [19]:
data['title'] = data['title'].apply(clean_all)

In [20]:
data['comment'] = data['comment'].apply(clean_all)

In [21]:
def convert_star(star): 
    if star in (1, 2): 
        return 'bad'
    elif star  == 3: 
        return 'neutral'
    return 'good'

In [22]:
data['label'] = data['star'].apply(convert_star)

In [23]:
filtered_df = data[data['comment'].str.strip() != '']
filtered_df = filtered_df[filtered_df['title'].str.strip() != '']

data = filtered_df

In [33]:
data

,category,sub_category,name_product,star,title,comment,label
0,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,nope buy bad quality,really want like problem order color option tw...,bad
1,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,buy,shirt look great picture wash seam start pull ...,bad
2,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,worst,bad item ever purchase son look cheap poor qua...,bad
3,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,happened,product definitely miss mark begin white shirt...,bad
4,Boys' Fashion,Clothing,HonestBaby Unisex Baby Organic Cotton Short Sl...,1,single stitch seam raw edge buy,single stitch seam video first wash never warn...,bad
...,...,...,...,...,...,...,...
45921,Women's Fashion,Accessories,"Eucerin Original Healing Cream, Fragrance Free...",5,love stuff,package came read two oz jar package seal nice...,good
45922,Women's Fashion,Accessories,"Eucerin Original Healing Cream, Fragrance Free...",5,healing moisture hospital use,use eucerine almost decade give late fatherinl...,good
45923,Women's Fashion,Accessories,"Eucerin Original Healing Cream, Fragrance Free...",5,best moisturizing cream,bad issue painful dry skin face start use nigh...,good
45924,Women's Fashion,Accessories,"Eucerin Original Healing Cream, Fragrance Free...",5,thick stay,deliver quick love stuff skin sensitive dry th...,good


In [ ]:
conn, cursor = f.init()

data.to_sql('REVIEWS_CLEANED', conn, if_exists='replace', index=False)
conn.close()